Używamy GAN.

Czemu GAN? Ponieważ ogólna zasada jest taka, że próbujemy coś wygenerować i mamy detykowanego agenta, który mówi czy to się udało czy nie.
Brzmi ok do uczenia generowania obrazów już istniejących.

W takim razie jak to działa że tworzy nowe obrazy?

Ano tak...

NIE WIEM ;___;